In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import random
    # to connect to mysql
import pandas as pd
import mysql.connector

# to push data into mysql
from tqdm import tqdm
import time
import numpy as np
import os
import re
import json
import numpy as np
# from langchain.embeddings import OpenAIEmbeddings
# import openai
# import nest_asyncio  # Import nest_asyncio
from fastapi import FastAPI, HTTPException, Request
from fastapi.middleware.cors import CORSMiddleware

from fastapi.responses import JSONResponse


# In[2]:

# nest_asyncio.apply()  # Apply nest_asyncio to allow asyncio.run() inside Jupyter


app = FastAPI()

allowed_origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=allowed_origins,
    allow_credentials=True,  # If you want to allow cookies to be sent
#    allow_methods=["*"],  # Specify the HTTP methods you want to allow
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],  # You can customize the allowed headers or use "*" to allow any header
)



def metadata_input(path = None):
    # Input DB Metadata / CSV 
    
    ### Rename To 'dbname', 'table_name', 'original_column_name', 'samples','data_dictionary_flag', 'data_tribe', 'column_description', 'max_value','min_value', 'n_unique', 'n_unique_per_row_count', 'pct_null_values','possible_values', 'sql_path', 'data_type'
    ### Convert all Columns that need to be vectorized and used as vector IDs into string for each row.
    import pandas as pd
    df =  pd.read_csv(path)
    df["table_name"] =  df['table_name'].astype(str)
    df["original_column_name"]=  df['original_column_name'].astype(str)
    df["column_description"]=  df['column_description'].astype(str)
    df["samples"]=  df['samples'].astype(str)
    
    df["non_null_fract"] = 1-df['pct_null_values']/100
    
    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, str) and x.strip() != x).any():
            pass
    
    def clean_unicode(text):
        try:
            # Try to decode the text as UTF-8
            cleaned_text = text.encode('utf-8').decode('utf-8')
        except UnicodeDecodeError:
            # If decoding fails, replace non-compliant characters with a replacement character
            cleaned_text = text.encode('utf-8', 'replace').decode('utf-8')
        return cleaned_text

    # Apply the cleaning function to the column
    df['original_column_name_unicode'] = df['original_column_name'].apply(clean_unicode)
    df['id'] = df['table_name']+ "." + df['original_column_name_unicode']
    df['id'] = df['id'].str.replace(' ', '|')  ##This is because, if ID has space in Pinecone "fetching_not possible"

    return df


@app.get("/DataObserAvinash_jsn")
async def main(query, databaseName:str,subjectIntent:str):
    #     val = "relationship_data.new_final_score_60"


    

    # print("databaseName")

    # print(databaseName)

    # print('subjectIntent')

    # print(subjectIntent)

    table_name_tmp:str

    indent:str

 

    

    array_param=str(query)

    table_name_tmp:str

    indent:str

 

    array=array_param.split(',')

       # print(type(array[0]))

    val=array[0]

    resultant_string = val[2:]

    resultant_string=resultant_string[:-1]

    print(resultant_string)

    table_name_tmp=resultant_string

    # array_Indent=array[1].split(':')

    # str_Indent=array_Indent[1]

    # str_Indent=str_Indent[:-4]

    # str_Indent=str_Indent[2:]

    # indent=str_Indent

    table_name_tmp=table_name_tmp.replace(" ","")

 

    # print("table_name_tmp: " + table_name_tmp)

    # print("indent: " + indent)

    # print(type(table_name_tmp))

    # print(type(indent))

    databaseName=databaseName.replace("+","")

    table_name_tmp=databaseName.replace(" ","")

    # if subjectIntent.find(':') != -1:

    #     array_Indent=subjectIntent.split(':')

 

    #     str_Indent  =array_Indent[1]

        

    #.replace('"','').strip()

    # else :

    #     str_Indent=subjectIntent

 

    # str_Indent=str_Indent.replace("+","").replace("}","")

    # indent=str_Indent.replace('\\',' ').replace(" ","")

    return table_name_tmp




   

 

   
    

 

    

  
 





import random
    # to connect to mysql
import pandas as pd
import mysql.connector

# to push data into mysql
import pymysql
import glob
from tqdm import tqdm
import time
import numpy as np
import os
import re
# import pinecone
import json
import numpy as np
# from langchain.embeddings import OpenAIEmbeddings
# import openai
# import nest_asyncio  # Import nest_asyncio
from fastapi import FastAPI, HTTPException, Request
from fastapi.middleware.cors import CORSMiddleware

from fastapi.responses import JSONResponse
# import chromadb
import json
import var
import chromadb
from chromadb.config import Settings

filename = 'config.json'

with open(filename, 'r') as file:
    config = json.load(file)
## password for sql
# password = urllib.parse.quote_plus(config['UserDataDB']['password'])
    
def call_chromadb():    

    username = var.CHROMA_DB_USERNAME
    password = var.CHROMA_DB_PASSWORD
    host = var.CHROMA_DB_HOST
    port = var.CHROMA_DB_PORT

    credentials = f"{username}:{password}"

    settings = Settings(chroma_client_auth_provider="chromadb.auth.basic_authn.BasicAuthClientProvider",
                        chroma_client_auth_credentials=credentials)

    # Connection
    chroma_client = chromadb.HttpClient(host=host, port=port, settings=settings)

    return chroma_client

def retrieve_data_from_chromadb(collection_name):
    client = call_chromadb()
    client = chromadb.PersistentClient(path=config['ChromaDB']['path'])
    collection = client.get_collection(collection_name.lower())
    results = collection.get()
    data = [doc for doc in results["metadatas"]]
    ids = results['ids']

    df = pd.DataFrame(data)
    df.index = ids
    df.index.name = 'Chroma_ID'

    return df


# In[2]:

# nest_asyncio.apply()  # Apply nest_asyncio to allow asyncio.run() inside Jupyter


app = FastAPI()

allowed_origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=allowed_origins,
    allow_credentials=True,  # If you want to allow cookies to be sent
#    allow_methods=["*"],  # Specify the HTTP methods you want to allow
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],  # You can customize the allowed headers or use "*" to allow any header
)



def metadata_input(path = None):
    # Input DB Metadata / CSV 
    
    ### Rename To 'dbname', 'table_name', 'original_column_name', 'samples','data_dictionary_flag', 'data_tribe', 'column_description', 'max_value','min_value', 'n_unique', 'n_unique_per_row_count', 'pct_null_values','possible_values', 'sql_path', 'data_type'
    ### Convert all Columns that need to be vectorized and used as vector IDs into string for each row.
    import pandas as pd
    df =  pd.read_csv(path)
    df["table_name"] =  df['table_name'].astype(str)
    df["original_column_name"]=  df['original_column_name'].astype(str)
    df["column_description"]=  df['column_description'].astype(str)
    df["samples"]=  df['samples'].astype(str)
    
    df["non_null_fract"] = 1-df['pct_null_values']/100
    
    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, str) and x.strip() != x).any():
            pass
    
    def clean_unicode(text):
        try:
            # Try to decode the text as UTF-8
            cleaned_text = text.encode('utf-8').decode('utf-8')
        except UnicodeDecodeError:
            # If decoding fails, replace non-compliant characters with a replacement character
            cleaned_text = text.encode('utf-8', 'replace').decode('utf-8')
        return cleaned_text

    # Apply the cleaning function to the column
    df['original_column_name_unicode'] = df['original_column_name'].apply(clean_unicode)
    df['id'] = df['table_name']+ "." + df['original_column_name_unicode']
    df['id'] = df['id'].str.replace(' ', '|')  ##This is because, if ID has space in Pinecone "fetching_not possible"

    return df




@app.get("/mek_str_2_jsn1")
async def main1(query, databaseName:str,subjectIntent:str):
#     val = "relationship_data.new_final_score_60"

#     val = "relationship_data.new_final_score_60"


    

    # print("databaseName")

    # print(databaseName)

    # print('subjectIntent')

    # print(subjectIntent)

    table_name_tmp:str

    indent:str

 

    

    array_param=str(query)

    table_name_tmp:str

    indent:str

 

    array=array_param.split(',')

       # print(type(array[0]))

    val=array[0]

    resultant_string = val[2:]

    resultant_string=resultant_string[:-1]

    print(resultant_string)

    table_name_tmp=resultant_string

    array_Indent=array[1].split(':')

    str_Indent=array_Indent[1]

    str_Indent=str_Indent[:-4]

    str_Indent=str_Indent[2:]

    indent=str_Indent

    table_name_tmp=table_name_tmp.replace(" ","")

 

    # print("table_name_tmp: " + table_name_tmp)

    # print("indent: " + indent)

    # print(type(table_name_tmp))

    # print(type(indent))

    databaseName=databaseName.replace("+","")

    table_name_tmp=databaseName.replace(" ","")

    if subjectIntent.find(':') != -1:

        array_Indent=subjectIntent.split(':')

 

        str_Indent  =array_Indent[1]

        

    #.replace('"','').strip()

    else :

        str_Indent=subjectIntent

 

    str_Indent=str_Indent.replace("+","").replace("}","")

    indent=str_Indent.replace('\\',' ').replace(" ","")
    
    
    print("hello")
    
@app.get("/")
async def main_test():
    return "Hello Hii"
     
@app.get("/mek_str_2_jsn")
async def main1(query, databaseName:str,subjectIntent:str):
    dataAssetName = databaseName.split(".")[1]
    print("Input", databaseName, dataAssetName)

    table_name_tmp:str

    indent:str

    array_param=str(query)

    table_name_tmp:str

    indent:str

    array=array_param.split(',')

    val=array[0]

    resultant_string = val[2:]

    resultant_string=resultant_string[:-1]

    table_name_tmp=resultant_string

    table_name_tmp=table_name_tmp.replace(" ","")


    databaseName=databaseName.replace("+","")

    table_name_tmp=databaseName.replace(" ","")

    val = table_name_tmp

    ######## Get relationship data from chroma ########
    collection_name = config['ChromaDB']['relationships_namespace']
    relevant_cols = ['DB_A', 'Table_A', 'Table_A_Column', 'DB_B', 'Table_B',
        'Table_B_Column', 'Flag_prim_table_A', 'Flag_prim_table_B',
        'Final_Score']
    df = retrieve_data_from_chromadb(collection_name)
    df = df.drop([col for col in df.columns if col not in relevant_cols], axis=1)

    ######## Get highlights from client_business_agent ########
    connection = mysql.connector.connect(
        host=config['DataAssetDB']['host'],
        user=config['DataAssetDB']['user'],
        password=config['DataAssetDB']['password']
        )

    cursor = connection.cursor()

    query2 = "SELECT * FROM datalake.dev_business_agent WHERE `Data_Asset_Name` = %s"
    try:
        cursor.execute(query2, (dataAssetName,))
        rows = cursor.fetchall()
        df2 = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
        highlighted_ids = df2['Database_Name'].apply(lambda x: x.split('.')[1]).tolist()
    except mysql.connector.Error as e:
        print(f"Error fetching data for second query: {e}")

    cursor.close()
    connection.close()

    ####### Import the Metadata ########
    # [CHANGE]: Get Metadata from chroma
    df_othr_info = metadata_input(path = "1_oct_metadata.csv")
    dftemp2 = pd.read_csv('metadat_latest_1oct.csv') #####
    dftemp2 = dftemp2[['table_name', 'source', 'table_description']].copy()
    df_other = pd.merge(df_othr_info, dftemp2, on='table_name',how = 'left')

    ###### Logic Open ########
    ####  Logic : Date Column Removal in all the RelationShip data, this can be ignored for the Next vesion of the Relationship data
    temp = df.copy()
    df_other_sub = df_other[["table_name","original_column_name","data_tribe","data_type", "table_description", "source"]].copy()
    result = pd.merge(temp, df_other_sub, left_on=['Table_A', 'Table_A_Column'],right_on=['table_name', 'original_column_name'], how='inner')
    condition = (result["data_tribe"] == 'date_related' ) | (result["data_type"] == 'datetime' )


    # Use the condition to filter the DataFrame
    filtered_df = result[~condition]
    filtered_df = filtered_df.drop(['data_tribe','data_type'], axis=1)



    #### Date Column Removal in all the RelationShip data, this can be ignored for the Next vesion of the Relationship data

    result = pd.merge(filtered_df, df_other_sub, left_on=['Table_B', 'Table_B_Column'],right_on=['table_name', 'original_column_name'], how='inner')
    condition = (result["data_tribe"] == 'date_related' ) | (result["data_type"] == 'datetime' )


    # Use the condition to filter the DataFrame
    filtered_df = result[~condition]
    filtered_df = filtered_df.drop(['data_tribe','data_type'], axis=1)
    ###### Logic Close  #########
    df = filtered_df.copy()

    seed_value = 42
    np.random.seed(seed_value)  # Set the random seed for NumPy

    # Get a random sample of one row from the DataFrame df with the specified seed
    random_row = df.sample(n=1, random_state=seed_value)

    #     # Select specific columns from the randomly selected row
    #     selected_columns = random_row[['DB_A', 'Table_A']]
    #     selected_columns = selected_columns.reset_index(drop=True)
    #     a = selected_columns.at[0, 'DB_A']
    #     b = selected_columns.at[0, 'Table_A']
    # #     db_tb_nam = a + "." + b                 ############################ Input Value here
    #     # db_tb_nam
    db_tb_nam = val
    left_part, right_part = db_tb_nam.split(".", 1)

    df_tmp_A = df[(df["DB_A"] ==  left_part) & (df["Table_A"] ==  right_part)]
    df_tmp_A.reset_index(drop=True,inplace = True)
    df_tmp_B = df[(df["DB_B"] ==  left_part) & (df["Table_B"] ==  right_part)]
    df_tmp_B.reset_index(drop=True,inplace = True)

    dct_a = {"DB_A": "DB_A_CH", "Table_A": "Table_A_CH","Table_A_Column" : "Table_A_Column_CH","Flag_prim_table_A" : "Flag_prim_table_A_CH"}
    df_tmp_B.rename(columns=dct_a, inplace=True)

    dct_b =  {"DB_B": "DB_A", "Table_B": "Table_A","Table_B_Column" : "Table_A_Column","Flag_prim_table_B" : "Flag_prim_table_A"}
    df_tmp_B.rename(columns=dct_b, inplace=True)

    dct_a_ch = {"DB_A_CH": "DB_B", "Table_A_CH": "Table_B","Table_A_Column_CH" : "Table_B_Column","Flag_prim_table_A_CH" : "Flag_prim_table_B"}
    df_tmp_B.rename(columns=dct_a_ch, inplace=True)

    df_required = pd.concat([df_tmp_A, df_tmp_B], ignore_index=True)
    df_required = df_required.sort_values(by='Final_Score',ascending = False).reset_index(drop=True)
    df_required = df_required[df_required["Final_Score"] >= 65] # [CHANGE]: should be 80


    ##### Logic Open #######
    ###### Combo Information for Each tAble ####3
    df_other["source"].fillna("Flat_File", inplace=True)
    df_other["source"].replace("SAP", "sap_combo", inplace=True)
    df_other["source"].replace("Flat_File", "flat_file_combo", inplace=True)
    df_other["source"].replace("data_asset_combo", "data_asset_combo", inplace=True)
    df_intm = df_other[["table_name","source"]].copy()
    df_intm = df_intm.drop_duplicates()

    combo_info = {row["table_name"]: row["source"] for index, row in df_intm.iterrows()} ## This is the Dictionary that has the Table Information as the Key and the Value as the Source File 
    ##### Logic Close #######

    ##### Logic Open Nodes#######
    dct = {"id": '',
    "label": '',
    "type": '',
    "comboId": ''}

    dct_col = {"id": '',
    "label": '',
    "data": { "type": '',"description": ''},
    "type": '',
    "comboId": ''}


    # [CHANGE]: quality is being hardcoded
    dct_lst = []
    jss = []
    dtst = df_required.loc[0,'Table_A']
    dt_dsc = df_required.loc[0, 'table_description_x']
    dt_src = df_required.loc[0, 'source_x']

    dct["id"] = dtst
    dct["label"] = dtst.upper()
    dct["type"] = 'table'
    dct["comboId"] = 'data_asset_combo'  ### Make Change Here When the required info comes
    dct["data"] = {"description": dt_dsc, 'source': "" if np.isnan(dt_src) else dt_src, 'quality': "100%"}
    dct["highlighted"] = True
    tmp = dct.copy()
    
    dct_lst.append(tmp)
    
    remn = df_required['Table_B'].unique()
    for i in remn.tolist():
        filtered_df = df_required[df_required['Table_B'] == i]

        # Assuming there's exactly one matching row for each unique Table_B value,
        # or taking the first one if there are multiple
        if not filtered_df.empty:
            dt_src = filtered_df.iloc[0]['source_y']
            dt_dsc = filtered_df.iloc[0]['table_description_y']

            dct = {}
            dct["id"] = i
            dct["label"] = i.upper()
            dct["type"] = 'table'
            dct["comboId"] = combo_info[i]  # Ensure combo_info is defined and has all necessary keys
            dct["data"] = {
                "description": dt_dsc,
                'source': "" if pd.isna(dt_src) else dt_src,
                'quality': "100%"
            }
            
            # Append a copy of the dictionary to the list to avoid overwriting
            dct_lst.append(dct.copy()) 



    df_unique = df_required.drop_duplicates(subset="Table_A_Column") 
    for index, row in df_unique.iterrows():
        dt_trb = df_other["data_tribe"][(df_other["table_name"] == row["Table_A"]) & (df_other["original_column_name"] == row["Table_A_Column"])].values[0]
        dt_dscr = df_other["column_description"][(df_other["table_name"] == row["Table_A"]) & (df_other["original_column_name"] == row["Table_A_Column"])].values[0]
        dt_src = df_other["source"][(df_other["table_name"] == row["Table_A"]) & (df_other["original_column_name"] == row["Table_A_Column"])].values[0]

        dictionary = {"id": row["Table_A"]+ "_"+ row["Table_A_Column"],
                "label": row["Table_A_Column"],
                "data": { "type": dt_trb,"description": dt_dscr, "source": dt_src, "quality": "100%"},
                "type": 'column',
                "comboId": "data_asset_combo"
        }
        dct_lst.append(dictionary)
    df_unique_b     = df_required.drop_duplicates(subset=['Table_B', 'Table_B_Column']) ### There might be repetation of the Same table and Same column name again
    for index, row in df_unique_b.iterrows():
        dt_trb = df_other["data_tribe"][(df_other["table_name"] == row["Table_B"]) & (df_other["original_column_name"] == row["Table_B_Column"])].values[0]
        dt_dscr = df_other["column_description"][(df_other["table_name"] == row["Table_B"]) & (df_other["original_column_name"] == row["Table_B_Column"])].values[0]
        dt_src = df_other["source"][(df_other["table_name"] == row["Table_B"]) & (df_other["original_column_name"] == row["Table_B_Column"])].values[0]

        dictionary = {"id": row["Table_B"]+ "_"+ row["Table_B_Column"],
                    "label": row["Table_B_Column"],
                    "data": { "type": dt_trb,"description": dt_dscr, "source": dt_src, "quality": "100%"},
                    "type": 'column',
                    "comboId": combo_info[row["Table_B"]]
        }
        dct_lst.append(dictionary)



    ##### Logic Close Nodes#######



    ##### Logic Open Edges#######
    table_a_col_a = []   
    df_unique = df_required.drop_duplicates(subset="Table_A_Column") 
    for index, row in df_unique.iterrows(): #### Here We are Connecting Table A to Columns A, When Taking the Unique the duplicate nodes are removed
        src = row["Table_A"]
        trgt = row["Table_A"]+"_"+row["Table_A_Column"]
        dictionary = {
            "source": src,
            "target": trgt
        } 
        table_a_col_a.append(dictionary)

    col_a_col_b = []
    for index, row in df_required.iterrows():  ### Here Connecting the Columns of A to Columns of B
        src = row["Table_A"]+"_"+row["Table_A_Column"]
        trgt = row["Table_B"]+"_"+row["Table_B_Column"]
        dictionary = {
            "source": src,
            "target": trgt
        }
        col_a_col_b.append(dictionary)

    table_b_col_b = []
    df_unique_b     = df_required.drop_duplicates(subset=['Table_B', 'Table_B_Column'])    
    for index, row in df_unique_b.iterrows():  ### Here Connecting the Tables of B to Columns of B 
        trgt = row["Table_B"]+"_"+row["Table_B_Column"]
        src = row["Table_B"]
        dictionary = {
            "source": src,
            "target": trgt
        }
        table_b_col_b.append(dictionary)
    
    # Building a mapping of columns to tables for lookup
    column_to_table_a = {d['target']: d['source'] for d in table_a_col_a}
    column_to_table_b = {d['target']: d['source'] for d in table_b_col_b}

    # Merge logic
    result_edge = []

    # For each colA to colB mapping
    for col_a_b in col_a_col_b:
        source_table = column_to_table_a.get(col_a_b['source'])
        target_table = column_to_table_b.get(col_a_b['target'])

        # Check if the table combination already exists in output
        existing_entry = next((item for item in result_edge if item['source'] == source_table and item['target'] == target_table), None)
        
        if existing_entry:
            # Append columns to the existing table mapping
            existing_entry['columns']['table_1'].append(col_a_b['source'])
            existing_entry['columns']['table_2'].append(col_a_b['target'])
        else:
            # Create new entry if it doesn't exist
            result_edge.append({
                'source': source_table,
                'target': target_table,
                'columns': {
                    'table_1': [col_a_b['source']],
                    'table_2': [col_a_b['target']]
                }
            })
    ##### Logic Close Edges #######

    ##### Logic Open Combos #######
    combos = [
                { "id": "data_asset_combo", "label": "Data_Asset","collapsed": False, "padding": [50,50,50, 50]},
                { "id": "sap_combo", "label": "SAP" ,"collapsed": False, "padding": [50,50,50, 50]},
                { "id": "flat_file_combo", "label": "Flat_File","collapsed": False, "padding": [50,50,50, 50]}
            ]

    ##### Logic Close Combos #######

    #### Remove columns from nodes ####
    # [CHANGE]: Should we really be doing this?
    dct_lst = [dct for dct in dct_lst if dct['type'] == 'table']

    #### Logic Add Highlights ####
    for item in dct_lst[1:]:
        item['highlighted'] = item['id'] in highlighted_ids
        # if item['comboId'] == 'data_asset_combo': 
        #     item['highlighted'] = True

    #### Logic Unite as one dictionary #####
    final_dct = {}
    final_dct["nodes"] = dct_lst
    final_dct["edges"] = result_edge
    final_dct["combos"] = combos   
    
    return final_dct



In [13]:
def main1(query, databaseName,subjectIntent):
    dataAssetName = databaseName.split(".")[1]
    print(dataAssetName)
    print("Input", databaseName, dataAssetName)

    table_name_tmp:str

    indent:str

    array_param=str(query)

    table_name_tmp:str

    indent:str

    array=array_param.split(',')

    val=array[0]

    resultant_string = val[2:]

    resultant_string=resultant_string[:-1]

    table_name_tmp=resultant_string

    table_name_tmp=table_name_tmp.replace(" ","")


    databaseName=databaseName.replace("+","")

    table_name_tmp=databaseName.replace(" ","")

    val = table_name_tmp

    # collection_name = config['ChromaDB']['relationships_namespace']
    relevant_cols = ['DB_A', 'Table_A', 'Table_A_Column', 'DB_B', 'Table_B',
        'Table_B_Column', 'Flag_prim_table_A', 'Flag_prim_table_B',
        'Final_Score']

    connection = mysql.connector.connect(
        host=config['DataAssetDB']['host'],
        user=config['DataAssetDB']['user'],
        password=config['DataAssetDB']['password']
        )

    cursor = connection.cursor()
    query1 = "SELECT * FROM relationship_data.relationship_metadata_211123"
    query2 = "SELECT * FROM datalake.dev_business_agent WHERE `Data_Asset_Name` = %s"
    try:
        cursor.execute(query2, ('bike',))
        rows = cursor.fetchall()
        df2 = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
        highlighted_ids = df2['Database_Name'].apply(lambda x: x.split('.')[1]).tolist()

        cursor.execute(query1)
        rows = cursor.fetchall()
        df1 = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
    except mysql.connector.Error as e:
        print(f"Error fetching data for second query: {e}")

    cursor.close()
    connection.close()
    ######## Get relationship data from chroma ########
    # collection_name = config['ChromaDB']['relationships_namespace']
    relevant_cols = ['DB_A', 'Table_A', 'Table_A_Column', 'DB_B', 'Table_B',
        'Table_B_Column', 'Flag_prim_table_A', 'Flag_prim_table_B',
        'Final_Score']
    
    df = df1
    df = df.drop([col for col in df.columns if col not in relevant_cols], axis=1)

    ######## Get highlights from client_business_agent ########

    ####### Import the Metadata ########
    # [CHANGE]: Get Metadata from chroma
    df_othr_info = metadata_input(path = "1_oct_metadata.csv")
    dftemp2 = pd.read_csv('metadat_latest_1oct.csv') #####
    dftemp2 = dftemp2[['table_name', 'source', 'table_description']].copy()
    df_other = pd.merge(df_othr_info, dftemp2, on='table_name',how = 'left')

    ###### Logic Open ########
    ####  Logic : Date Column Removal in all the RelationShip data, this can be ignored for the Next vesion of the Relationship data
    temp = df.copy()
    df_other_sub = df_other[["table_name","original_column_name","data_tribe","data_type", "table_description", "source"]].copy()
    result = pd.merge(temp, df_other_sub, left_on=['Table_A', 'Table_A_Column'],right_on=['table_name', 'original_column_name'], how='inner')
    condition = (result["data_tribe"] == 'date_related' ) | (result["data_type"] == 'datetime' )


    # Use the condition to filter the DataFrame
    filtered_df = result[~condition]
    filtered_df = filtered_df.drop(['data_tribe','data_type'], axis=1)

    #### Date Column Removal in all the RelationShip data, this can be ignored for the Next vesion of the Relationship data

    result = pd.merge(filtered_df, df_other_sub, left_on=['Table_B', 'Table_B_Column'],right_on=['table_name', 'original_column_name'], how='inner')
    condition = (result["data_tribe"] == 'date_related' ) | (result["data_type"] == 'datetime' )


    # Use the condition to filter the DataFrame
    filtered_df = result[~condition]
    filtered_df = filtered_df.drop(['data_tribe','data_type'], axis=1)
    ###### Logic Close  #########
    df = filtered_df.copy()

    seed_value = 42
    np.random.seed(seed_value)  # Set the random seed for NumPy

    # Get a random sample of one row from the DataFrame df with the specified seed
    random_row = df.sample(n=1, random_state=seed_value)

    #     # Select specific columns from the randomly selected row
    #     selected_columns = random_row[['DB_A', 'Table_A']]
    #     selected_columns = selected_columns.reset_index(drop=True)
    #     a = selected_columns.at[0, 'DB_A']
    #     b = selected_columns.at[0, 'Table_A']
    # #     db_tb_nam = a + "." + b                 ############################ Input Value here
    #     # db_tb_nam
    db_tb_nam = val
    left_part, right_part = db_tb_nam.split(".", 1)

    df_tmp_A = df[(df["DB_A"] ==  left_part) & (df["Table_A"] ==  right_part)]
    df_tmp_A.reset_index(drop=True,inplace = True)
    df_tmp_B = df[(df["DB_B"] ==  left_part) & (df["Table_B"] ==  right_part)]
    df_tmp_B.reset_index(drop=True,inplace = True)

    dct_a = {"DB_A": "DB_A_CH", "Table_A": "Table_A_CH","Table_A_Column" : "Table_A_Column_CH","Flag_prim_table_A" : "Flag_prim_table_A_CH"}
    df_tmp_B.rename(columns=dct_a, inplace=True)

    dct_b =  {"DB_B": "DB_A", "Table_B": "Table_A","Table_B_Column" : "Table_A_Column","Flag_prim_table_B" : "Flag_prim_table_A"}
    df_tmp_B.rename(columns=dct_b, inplace=True)

    dct_a_ch = {"DB_A_CH": "DB_B", "Table_A_CH": "Table_B","Table_A_Column_CH" : "Table_B_Column","Flag_prim_table_A_CH" : "Flag_prim_table_B"}
    df_tmp_B.rename(columns=dct_a_ch, inplace=True)

    df_required = pd.concat([df_tmp_A, df_tmp_B], ignore_index=True)
    df_required = df_required.sort_values(by='Final_Score',ascending = False).reset_index(drop=True)
    df_required = df_required[df_required["Final_Score"] >= 65] # [CHANGE]: should be 80


    ##### Logic Open #######
    ###### Combo Information for Each tAble ####3
    df_other["source"].fillna("Flat_File", inplace=True)
    df_other["source"].replace("SAP", "sap_combo", inplace=True)
    df_other["source"].replace("Flat_File", "flat_file_combo", inplace=True)
    df_other["source"].replace("data_asset_combo", "data_asset_combo", inplace=True)
    df_intm = df_other[["table_name","source"]].copy()
    df_intm = df_intm.drop_duplicates()

    combo_info = {row["table_name"]: row["source"] for index, row in df_intm.iterrows()} ## This is the Dictionary that has the Table Information as the Key and the Value as the Source File 
    ##### Logic Close #######

    ##### Logic Open Nodes#######
    dct = {"id": '',
    "label": '',
    "type": '',
    "comboId": ''}

    dct_col = {"id": '',
    "label": '',
    "data": { "type": '',"description": ''},
    "type": '',
    "comboId": ''}


    # [CHANGE]: quality is being hardcoded
    dct_lst = []
    jss = []
    dtst = df_required.loc[0,'Table_A']
    dt_dsc = df_required.loc[0, 'table_description_x']
    dt_src = df_required.loc[0, 'source_x']

    dct["id"] = dtst
    dct["label"] = dtst.upper()
    dct["type"] = 'table'
    dct["comboId"] = 'data_asset_combo'  ### Make Change Here When the required info comes
    dct["data"] = {"description": dt_dsc, 'source': "" if np.isnan(dt_src) else dt_src, 'quality': "100%"}
    dct["highlighted"] = True
    tmp = dct.copy()
    
    dct_lst.append(tmp)
    
    remn = df_required['Table_B'].unique()
    for i in remn.tolist():
        filtered_df = df_required[df_required['Table_B'] == i]

        # Assuming there's exactly one matching row for each unique Table_B value,
        # or taking the first one if there are multiple
        if not filtered_df.empty:
            dt_src = filtered_df.iloc[0]['source_y']
            dt_dsc = filtered_df.iloc[0]['table_description_y']

            dct = {}
            dct["id"] = i
            dct["label"] = i.upper()
            dct["type"] = 'table'
            dct["comboId"] = combo_info[i]  # Ensure combo_info is defined and has all necessary keys
            dct["data"] = {
                "description": dt_dsc,
                'source': "" if pd.isna(dt_src) else dt_src,
                'quality': "100%"
            }
            
            # Append a copy of the dictionary to the list to avoid overwriting
            dct_lst.append(dct.copy()) 



    df_unique = df_required.drop_duplicates(subset="Table_A_Column") 
    for index, row in df_unique.iterrows():
        dt_trb = df_other["data_tribe"][(df_other["table_name"] == row["Table_A"]) & (df_other["original_column_name"] == row["Table_A_Column"])].values[0]
        dt_dscr = df_other["column_description"][(df_other["table_name"] == row["Table_A"]) & (df_other["original_column_name"] == row["Table_A_Column"])].values[0]
        dt_src = df_other["source"][(df_other["table_name"] == row["Table_A"]) & (df_other["original_column_name"] == row["Table_A_Column"])].values[0]

        dictionary = {"id": row["Table_A"]+ "_"+ row["Table_A_Column"],
                "label": row["Table_A_Column"],
                "data": { "type": dt_trb,"description": dt_dscr, "source": dt_src, "quality": "100%"},
                "type": 'column',
                "comboId": "data_asset_combo"
        }
        dct_lst.append(dictionary)
    df_unique_b     = df_required.drop_duplicates(subset=['Table_B', 'Table_B_Column']) ### There might be repetation of the Same table and Same column name again
    for index, row in df_unique_b.iterrows():
        dt_trb = df_other["data_tribe"][(df_other["table_name"] == row["Table_B"]) & (df_other["original_column_name"] == row["Table_B_Column"])].values[0]
        dt_dscr = df_other["column_description"][(df_other["table_name"] == row["Table_B"]) & (df_other["original_column_name"] == row["Table_B_Column"])].values[0]
        dt_src = df_other["source"][(df_other["table_name"] == row["Table_B"]) & (df_other["original_column_name"] == row["Table_B_Column"])].values[0]

        dictionary = {"id": row["Table_B"]+ "_"+ row["Table_B_Column"],
                    "label": row["Table_B_Column"],
                    "data": { "type": dt_trb,"description": dt_dscr, "source": dt_src, "quality": "100%"},
                    "type": 'column',
                    "comboId": combo_info[row["Table_B"]]
        }
        dct_lst.append(dictionary)



    ##### Logic Close Nodes#######



    ##### Logic Open Edges#######
    table_a_col_a = []   
    df_unique = df_required.drop_duplicates(subset="Table_A_Column") 
    for index, row in df_unique.iterrows(): #### Here We are Connecting Table A to Columns A, When Taking the Unique the duplicate nodes are removed
        src = row["Table_A"]
        trgt = row["Table_A"]+"_"+row["Table_A_Column"]
        dictionary = {
            "source": src,
            "target": trgt
        } 
        table_a_col_a.append(dictionary)

    col_a_col_b = []
    for index, row in df_required.iterrows():  ### Here Connecting the Columns of A to Columns of B
        src = row["Table_A"]+"_"+row["Table_A_Column"]
        trgt = row["Table_B"]+"_"+row["Table_B_Column"]
        dictionary = {
            "source": src,
            "target": trgt
        }
        col_a_col_b.append(dictionary)

    table_b_col_b = []
    df_unique_b     = df_required.drop_duplicates(subset=['Table_B', 'Table_B_Column'])    
    for index, row in df_unique_b.iterrows():  ### Here Connecting the Tables of B to Columns of B 
        trgt = row["Table_B"]+"_"+row["Table_B_Column"]
        src = row["Table_B"]
        dictionary = {
            "source": src,
            "target": trgt
        }
        table_b_col_b.append(dictionary)
    
    # Building a mapping of columns to tables for lookup
    column_to_table_a = {d['target']: d['source'] for d in table_a_col_a}
    column_to_table_b = {d['target']: d['source'] for d in table_b_col_b}

    # Merge logic
    result_edge = []

    # For each colA to colB mapping
    for col_a_b in col_a_col_b:
        source_table = column_to_table_a.get(col_a_b['source'])
        target_table = column_to_table_b.get(col_a_b['target'])

        # Check if the table combination already exists in output
        existing_entry = next((item for item in result_edge if item['source'] == source_table and item['target'] == target_table), None)
        
        if existing_entry:
            # Append columns to the existing table mapping
            existing_entry['columns']['table_1'].append(col_a_b['source'])
            existing_entry['columns']['table_2'].append(col_a_b['target'])
        else:
            # Create new entry if it doesn't exist
            result_edge.append({
                'source': source_table,
                'target': target_table,
                'columns': {
                    'table_1': [col_a_b['source']],
                    'table_2': [col_a_b['target']]
                }
            })
    ##### Logic Close Edges #######

    ##### Logic Open Combos #######
    combos = [
                { "id": "data_asset_combo", "label": "Data_Asset","collapsed": False, "padding": [50,50,50, 50]},
                { "id": "sap_combo", "label": "SAP" ,"collapsed": False, "padding": [50,50,50, 50]},
                { "id": "flat_file_combo", "label": "Flat_File","collapsed": False, "padding": [50,50,50, 50]}
            ]

    ##### Logic Close Combos #######

    #### Remove columns from nodes ####
    # [CHANGE]: Should we really be doing this?
    dct_lst = [dct for dct in dct_lst if dct['type'] == 'table']

    #### Logic Add Highlights ####
    for item in dct_lst[1:]:
        item['highlighted'] = item['id'] in highlighted_ids
        # if item['comboId'] == 'data_asset_combo': 
        #     item['highlighted'] = True

    #### Logic Unite as one dictionary #####
    final_dct = {}
    final_dct["nodes"] = dct_lst
    final_dct["edges"] = result_edge
    final_dct["combos"] = combos   
    
    return final_dct



In [14]:
query = "give me bike sales data"
databaseName = "datalake.bike"
subjectIntent = "give me bike sales data"
df  = main1(query, databaseName,subjectIntent)

bike
Input datalake.bike bike


ValueError: a must be greater than 0 unless no samples are taken

In [12]:
df

{'nodes': [{'id': 'bike',
   'label': 'BIKE',
   'type': 'table',
   'comboId': 'data_asset_combo',
   'data': {'description': 'A fact table containing information about bike sales transactions, such as transaction ID, product ID, customer ID, transaction date, online order status, brand, product line, product class, product size, list price, standard cost, and product first sold date. It is used to track and analyze sales data for reporting, analysis, and auditing purposes.',
    'source': '',
    'quality': '100%'},
   'highlighted': True},
  {'id': 'expense_data_spend_analysis',
   'label': 'EXPENSE_DATA_SPEND_ANALYSIS',
   'type': 'table',
   'comboId': 'flat_file_combo',
   'data': {'description': "A fact table containing detailed information about financial expenses, such as fiscal year period, service, department, program, fund, category, expense category, account code, invoice description, vendor, vendor number, invoice number, invoice line, invoice distribution line, amount, i

In [8]:
connection = mysql.connector.connect(
        host=config['DataAssetDB']['host'],
        user=config['DataAssetDB']['user'],
        password=config['DataAssetDB']['password']
        )

cursor = connection.cursor()
query1 = "SELECT * FROM relationship_data.relationship_metadata_211123"
query2 = "SELECT * FROM datalake.dev_business_agent WHERE `Data_Asset_Name` = %s"
try:
    cursor.execute(query2, ('bike',))
    rows = cursor.fetchall()
    df2 = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
    highlighted_ids = df2['Database_Name'].apply(lambda x: x.split('.')[1]).tolist()

    cursor.execute(query1)
    rows = cursor.fetchall()
    df1 = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
except mysql.connector.Error as e:
    print(f"Error fetching data for second query: {e}")

cursor.close()
connection.close()

In [9]:
df1

,DB_A,Table_A,Table_A_Column,DB_B,Table_B,Table_B_Column,Flag_prim_table_A,Flag_prim_table_B,Final_Score,Sample_Sim,ColName_Sim,ColDsc_Sim,Euc_Sim,Geo_Mean,Match_Ratio_AB,Match_Ratio_BA,Flag_FK
0,datalake,account_receivable_with_dimention,cust_number,datalake,bank_transactions,CustAccountBalance,None,None,58,0.83,0.81,0.86,0.83,0.83,0.010638,0.004739,0
1,datalake,account_receivable_with_dimention,cust_number,datalake,bank_transactions,TransactionAmount (INR),None,None,55,0.82,0.78,0.76,0.77,0.79,0.010638,0.004739,0
2,datalake,account_receivable_with_dimention,cust_number,datalake,banks_reviews,like,None,None,54,0.82,0.68,0.76,0.72,0.77,0.000000,0.000000,0
3,datalake,account_receivable_with_dimention,cust_number,datalake,bike,customer_id,None,None,70,0.85,0.85,0.89,0.87,0.86,0.000000,0.000000,0
4,datalake,account_receivable_with_dimention,cust_number,datalake,bike,list_price,None,None,55,0.81,0.76,0.74,0.75,0.78,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3179652,data_warehouse,cost,TKE001,data_warehouse,tvst,SQL_CHG_VERSION,None,None,55,0.76,0.67,0.65,0.66,0.71,0.166667,1.000000,0
3179653,data_warehouse,cost,TKE001,data_warehouse,t683u,SQL_CHG_VERSION,None,None,55,0.76,0.67,0.65,0.66,0.71,0.166667,1.000000,0
3179654,data_warehouse,cost,TKE001,data_warehouse,tvtkt,SQL_CHG_OP_CD,None,None,48,0.70,0.68,0.66,0.67,0.68,0.000000,0.000000,0
3179655,data_warehouse,cost,TKE001,data_warehouse,tvtkt,SQL_CHG_VERSION,None,None,55,0.76,0.67,0.65,0.66,0.71,0.166667,1.000000,0


In [7]:
highlighted_ids

['customer_master_data', 'expense_data_spend_analysis', 'house_prices_v2']